In [94]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("upper_case", return_direct=True)
def to_upper_case(input:str) -> str:
  """Returns the input as all upper case."""
  return input.upper()

@tool("random_number", return_direct=True)
def random_number_maker() -> int:
    """Returns a random float number between 0-100."""
    # return random.randint(0, 100)
    return round(random.randint(0, 100) + random.random(),2)

tools = [to_lower_case,to_upper_case,random_number_maker]

from langchain import hub
 # prompt = hub.pull("anthonydresser/structured-chat-agent-llama")
prompt = hub.pull("hwchase17/openai-functions-agent")

c:\Users\panya\anaconda3\envs\chat\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [93]:
round(random.randint(0, 100) + random.random(),2)

25.22

In [24]:
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [95]:
from langchain_ollama import ChatOllama
# llm = ChatOllama(model="llama3.2:latest", temperature=0)
# llm = ChatOllama(model="llama3.1:latest", temperature=0)
llm = ChatOllama(model="qwen2.5:7b", temperature=0.1)
llm_with_tools = llm.bind_tools(tools)

In [96]:
#  test ChatPromptTemplate
from langchain.prompts.chat import (
ChatPromptTemplate,
SystemMessagePromptTemplate,
AIMessagePromptTemplate,
HumanMessagePromptTemplate,)
template="You are a helpful assistant who can give {category} for given input. Only give {category} no other text, use as little words as possible while responding, better give reply only in one word."
system_message_prompt=SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

t = chat_prompt.format_prompt(category="antonyms", text="Happy").to_messages()
llm.invoke(t)


AIMessage(content='Sad', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T17:15:55.7929356Z', 'done': True, 'done_reason': 'stop', 'total_duration': 113483900, 'load_duration': 17435200, 'prompt_eval_count': 53, 'prompt_eval_duration': 17000000, 'eval_count': 2, 'eval_duration': 71000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5eb33948-d697-4cf6-a457-826ce920a64f-0', usage_metadata={'input_tokens': 53, 'output_tokens': 2, 'total_tokens': 55})

In [97]:
a = prompt.format_prompt(
    input="give me a random number and then write in words and make it lower case.", 
    chat_history=[],
    agent_scratchpad=[]
    ).to_messages()

# simple

In [98]:
from langgraph.graph import MessagesState
from langgraph.graph import START,END, StateGraph
from langgraph.prebuilt import ToolNode,tools_condition

tool_executor = ToolNode(tools)
# Node
def reasoner(state: MessagesState):
   return {"messages": [llm_with_tools.invoke(["You are a helpful assistant. result must be number in word"] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Add nodes
builder.add_node("reasoner", reasoner)
builder.add_node("tools", ToolNode(tools)) # for the tools

# Add edges
builder.add_edge(START, "reasoner")
builder.add_conditional_edges(
    "reasoner",
    # If the latest message (result) from node reasoner is a tool call -> tools_condition routes to tools
    # If the latest message (result) from node reasoner is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "reasoner")
app = builder.compile()
# app

In [99]:
inputs = {"messages": "give me a random number and then write in words and make it lower case."}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T17:16:01.0537722Z', 'done': True, 'done_reason': 'stop', 'total_duration': 611958400, 'load_duration': 18333500, 'prompt_eval_count': 276, 'prompt_eval_duration': 59000000, 'eval_count': 16, 'eval_duration': 530000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a924d6cd-61ec-4610-ae02-93ddd12e9993-0', tool_calls=[{'name': 'random_number', 'args': {}, 'id': '732dcaee-a5a4-4666-8a07-124db253743e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 276, 'output_tokens': 16, 'total_tokens': 292})]}
----
{'messages': [ToolMessage(content='69.91', name='random_number', id='98105e50-0111-4c4d-b935-a7bd16c80917', tool_call_id='732dcaee-a5a4-4666-8a07-124db253743e')]}
----
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T17:16:01.6667243Z', 'done

In [100]:
inputs = {"messages": "give me a random number and then write in lower case words."}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T17:16:59.745471Z', 'done': True, 'done_reason': 'stop', 'total_duration': 617988000, 'load_duration': 17011400, 'prompt_eval_count': 273, 'prompt_eval_duration': 142000000, 'eval_count': 16, 'eval_duration': 451000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5a99bb97-a84a-467c-a3e8-4be196d1f67e-0', tool_calls=[{'name': 'random_number', 'args': {}, 'id': '6a6014de-2293-4806-9010-8716bc6c63f7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 273, 'output_tokens': 16, 'total_tokens': 289})]}
----
{'messages': [ToolMessage(content='38.05', name='random_number', id='1c0cf78a-3d4a-4f02-a9e2-ec981cbcd6c2', tool_call_id='6a6014de-2293-4806-9010-8716bc6c63f7')]}
----
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T17:17:00.3462705Z', 'done

In [102]:
inputs = {"messages": "give me a random number and then write in upper case words."}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T17:17:15.1921723Z', 'done': True, 'done_reason': 'stop', 'total_duration': 697404300, 'load_duration': 17978700, 'prompt_eval_count': 273, 'prompt_eval_duration': 8000000, 'eval_count': 16, 'eval_duration': 665000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-8fd566e1-5aaf-4a55-bff2-4e43a679e211-0', tool_calls=[{'name': 'random_number', 'args': {}, 'id': 'ab6fd9ec-8dfb-4e63-bbe8-cabd1a027b20', 'type': 'tool_call'}], usage_metadata={'input_tokens': 273, 'output_tokens': 16, 'total_tokens': 289})]}
----
{'messages': [ToolMessage(content='59.26', name='random_number', id='4e47f13d-55b8-4a7a-864c-0bd302dca7e7', tool_call_id='ab6fd9ec-8dfb-4e63-bbe8-cabd1a027b20')]}
----
{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T17:17:15.8395859Z', 'done'

# Class

In [21]:
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)
tool_executor

C:\Users\panya\AppData\Local\Temp\ipykernel_7208\3331265696.py:3: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


_execute(tags=None, recurse=True, func_accepts_config=True, func_accepts={'writer': False, 'store': False}, tools=[StructuredTool(name='lower_case', description='Returns the input as all lower case.', args_schema=<class 'langchain_core.utils.pydantic.lower_case'>, return_direct=True, func=<function to_lower_case at 0x000002263A376340>), StructuredTool(name='random_number', description='Returns a random number between 0-100.', args_schema=<class 'langchain_core.utils.pydantic.random_number'>, return_direct=True, func=<function random_number_maker at 0x000002263A375DA0>)], tool_map={'lower_case': StructuredTool(name='lower_case', description='Returns the input as all lower case.', args_schema=<class 'langchain_core.utils.pydantic.lower_case'>, return_direct=True, func=<function to_lower_case at 0x000002263A376340>), 'random_number': StructuredTool(name='random_number', description='Returns a random number between 0-100.', args_schema=<class 'langchain_core.utils.pydantic.random_number'>,

In [6]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [7]:
# Define the agent/graph
def run_agent(state: AgentState):
    print(state)
    # print(state)
    message = prompt.format_prompt(
        input=state['input'], 
        chat_history=state['chat_history'],
        agent_scratchpad=[]
    ).to_messages()
    agent_outcome = llm_with_tools.invoke(message)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(state: AgentState):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = state['agent_outcome']
    # Execute the tool
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    # Return the output
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(state: AgentState):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(state['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

In [8]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [9]:
workflow.channels

{'input': <langgraph.channels.last_value.LastValue at 0x262862c34c0>,
 'chat_history': <langgraph.channels.last_value.LastValue at 0x262862c3400>,
 'agent_outcome': <langgraph.channels.last_value.LastValue at 0x262862c3440>,
 'intermediate_steps': <langgraph.channels.binop.BinaryOperatorAggregate at 0x262861e97c0>}

In [10]:
inputs = {
    "input": "give me a random number and then write in words and make it lower case.", 
    "chat_history": [],
    # "agent_scratchpad" : []
    }
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'input': 'give me a random number and then write in words and make it lower case.', 'chat_history': [], 'intermediate_steps': []}
{'agent_outcome': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T16:46:36.7494865Z', 'done': True, 'done_reason': 'stop', 'total_duration': 772764800, 'load_duration': 18392000, 'prompt_eval_count': 212, 'prompt_eval_duration': 163000000, 'eval_count': 16, 'eval_duration': 587000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-7f1f376c-43ff-42ea-9d80-7512c5b92e52-0', tool_calls=[{'name': 'random_number', 'args': {}, 'id': '112581eb-35a3-4ec5-9b02-654751f63e6d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 212, 'output_tokens': 16, 'total_tokens': 228})}
----


AttributeError: 'AIMessage' object has no attribute 'tool'

In [16]:
    message = prompt.format_prompt(
        input=inputs['input'], 
        chat_history=inputs['chat_history'],
        agent_scratchpad=[]
    ).to_messages()
    agent_outcome = llm_with_tools.invoke(message)

In [17]:
agent_outcome

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-01-16T16:47:29.0864154Z', 'done': True, 'done_reason': 'stop', 'total_duration': 724778600, 'load_duration': 17363100, 'prompt_eval_count': 212, 'prompt_eval_duration': 265000000, 'eval_count': 16, 'eval_duration': 439000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-39e1772f-d109-4574-bc3f-e7ee98d52aff-0', tool_calls=[{'name': 'random_number', 'args': {}, 'id': 'fea2228b-2acd-4e0e-95e3-1799ec250430', 'type': 'tool_call'}], usage_metadata={'input_tokens': 212, 'output_tokens': 16, 'total_tokens': 228})